In [5]:
import pickle
import os

In [50]:
%load_ext autoreload
%autoreload 2

In [53]:
import common
from utils.nlp_utils import NLPUtils
from utils.io_utils import IOUtils

In [79]:
class Data:
    def __init__(self):
        self.w2i = None
        self.entries = None
        self.train_entries = None
        self.test_entries = None
        self.ext_embedding = None
        self.reviews = None
        self.predicted_reviews = None

    def to(self, device):
        if self.entries:
            for entry in self.entries:
                entry.index_tensor = entry.index_tensor.to(device=device)
        if self.reviews:
            for doc_id in self.reviews:
                for review in self.reviews[doc_id]:
                    review.index_tensor = review.index_tensor.to(device=device)
        if self.predicted_reviews:
            for doc_id in self.predicted_reviews:
                for review in self.predicted_reviews[doc_id]:
                    review.index_tensor = review.index_tensor.to(device=device)

    def load(self, infile):
        with open(infile, "rb") as target:
            self.ext_embeddings, self.entries, self.w2i = pickle.load(target)

    def save_data(self, infile):
        with open(infile, "wb") as target:
            pickle.dump([self.ext_embeddings, self.entries, self.w2i], target)

    def load_predicted_reviews(self, infile):
        with open(infile, "rb") as target:
            self.predicted_reviews = pickle.load(target)
        for app_id in self.predicted_reviews.keys():
            self.predicted_reviews[app_id].sort(
                key=lambda x: x.prediction_result.item(), reverse=True
            )

    def load_reviews(self, infile):
        with open(infile, "rb") as target:
            self.reviews = pickle.load(target)

In [7]:
saved_data = os.path.join(os.environ["SECURITY_DATASETS"], "saved-parameters/saved-data/emdeddings-sentences-w2i.pickle")

In [80]:
data = Data()
data.load(saved_data)

In [37]:
fasttext = IOUtils.load_embeddings_file("cc.en.300.bin", "fasttext", True)

In [ ]:
fasttext, dim = fasttext

In [81]:
ext_embeddings = {}

In [82]:
for entry in data.entries:
    preprocessed = NLPUtils.preprocess_sentence(entry.sentence, "NONE")
    entry.preprocessed_sentence = [word for word in preprocessed if word in fasttext]
    for word in entry.preprocessed_sentence:
        ext_embeddings[word] = fasttext[word]


In [83]:
data.ext_embeddings = ext_embeddings

In [88]:
w2i = {key:idx for idx,key in enumerate(list(ext_embeddings.keys()))}

In [90]:
data.w2i = w2i

In [91]:
fasttext_saved_data = os.path.join(os.environ["SECURITY_DATASETS"], "saved-parameters/saved-data/fasttext_emdeddings-sentences-w2i.pickle")

In [92]:
data.save_data(fasttext_saved_data)

In [86]:
len(fasttext)

1627124